In [1]:
import pandas as pd
import numpy as np
import re
from bigdata_a3_utils import load_compressed_dataset

c:\Users\annik\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
review_dataset = load_compressed_dataset(r"C:\Users\annik\Downloads\review\raw_review_Electronics.tar.gz")
meta_dataset = load_compressed_dataset(r"C:\Users\annik\Downloads\review\raw_meta_Electronics.tar.gz")

df_reviews = review_dataset["full"].to_pandas()
df_meta = meta_dataset["full"].to_pandas()
print("Loaded to pandas")

Extracting C:\Users\annik\Downloads\review\raw_review_Electronics.tar.gz to C:\Users\annik\Downloads\review\temp_a227debee69b4c52bf37954d9b96c8a3...
Loading dataset from C:\Users\annik\Downloads\review\temp_a227debee69b4c52bf37954d9b96c8a3\raw_review_Electronics...
Cleaning up temporary directory: C:\Users\annik\Downloads\review\temp_a227debee69b4c52bf37954d9b96c8a3
Extracting C:\Users\annik\Downloads\review\raw_meta_Electronics.tar.gz to C:\Users\annik\Downloads\review\temp_47288a473c5749cca1afe1b8774b585c...
Loading dataset from C:\Users\annik\Downloads\review\temp_47288a473c5749cca1afe1b8774b585c\raw_meta_Electronics...
Cleaning up temporary directory: C:\Users\annik\Downloads\review\temp_47288a473c5749cca1afe1b8774b585c
Loaded to pandas


In [3]:
# Replace known complex columns with NaN
complex_cols = ['images_x', 'images_y', 'videos', 'features', 'description', 'categories']
for col in complex_cols:
    if col in df_reviews.columns:
        df_reviews[col] = np.nan
    if col in df_meta.columns:
        df_meta[col] = np.nan

print("Complex columns set to NaN")

Complex columns set to NaN


In [4]:
import dask.dataframe as dd

In [7]:
import json
import numpy as np

df_reviews["images"] = df_reviews["images"].apply(
    lambda x: json.dumps(x.tolist()) if isinstance(x, np.ndarray)
    else json.dumps(x) if not pd.isna(x)
    else json.dumps(None)
)

In [9]:
import base64
import json

df_reviews["images"] = df_reviews["images"].apply(
    lambda x: base64.b64encode(x).decode('utf-8') if isinstance(x, np.ndarray) else json.dumps(x) if not pd.isna(x) else json.dumps(None)
)

In [ ]:
import base64
import pandas as pd
import numpy as np
import json

# Assuming 'df_meta' contains the images column
def encode_image(x):
    if isinstance(x, np.ndarray):
        return base64.b64encode(x).decode('utf-8')  # Convert ndarray to Base64 string
    return x if pd.notna(x) else None  # Handle NaN values or other cases

df_meta["images"] = df_meta["images"].apply(encode_image)

# Now you can safely save to Parquet
df_meta.to_parquet("meta.parquet", engine="fastparquet", index=False)

In [14]:
import base64
import pandas as pd
import numpy as np
import json

# Assuming 'df_reviews' contains the images
df_reviews["images"] = df_reviews["images"].apply(
    lambda x: base64.b64encode(x).decode('utf-8') if isinstance(x, np.ndarray) else json.dumps(x) if not pd.isna(x) else json.dumps(None)
)

# Now you can safely save to Parquet
df_reviews.to_parquet("reviews.parquet", engine="fastparquet", index=False)

In [18]:
import base64
import pandas as pd
import numpy as np
import json

# Function to handle the encoding of 'images' column
def encode_image(x):
    if isinstance(x, dict):  # Check if it's a dictionary (which we expect in 'images')
        # Convert the dictionary to a JSON string for serialization
        return json.dumps(x, default=str)  # Convert dictionary to a string
    return x  # For any other case, return the value as is

# Apply the encoding function to the 'images' column
df_meta["images"] = df_meta["images"].apply(encode_image)

# Verify the transformation by checking the first few rows
print(df_meta.head())

# Now save the DataFrame to Parquet
df_meta.to_parquet("meta.parquet", engine="fastparquet", index=False)

               main_category  \
0            All Electronics   
1            All Electronics   
2                  Computers   
3             AMAZON FASHION   
4  Cell Phones & Accessories   

                                               title  average_rating  \
0             FS-1051 FATSHARK TELEPORTER V3 HEADSET             3.5   
1                      Ce-H22B12-S1 4Kx2K Hdmi 4Port             5.0   
2  Digi-Tatoo Decal Skin Compatible With MacBook ...             4.5   
3  NotoCity Compatible with Vivoactive 4 band 22m...             4.5   
4             Motorola Droid X Essentials Combo Pack             3.8   

   rating_number  features  description  price  \
0              6       NaN          NaN   None   
1              1       NaN          NaN   None   
2            246       NaN          NaN  19.99   
3            233       NaN          NaN   9.99   
4             64       NaN          NaN  14.99   

                                              images  videos       store 

**Start Running From Here After Downloading The .parquet Files**

In [1]:
import dask.dataframe as dd
df_reviews_dd = dd.read_parquet("reviews.parquet")
df_meta_dd = dd.read_parquet("meta.parquet")

In [2]:
merged_dd = df_reviews_dd.merge(df_meta_dd, how='left', on='parent_asin')

In [3]:
# Filter ratings and non-empty text
filtered_dd = merged_dd[merged_dd['rating'].between(1, 5)]
filtered_dd = filtered_dd[filtered_dd['text'].notnull() & (filtered_dd['text'].str.strip() != "")]

# Trigger computation for filtered data
remaining_rows = filtered_dd.shape[0].compute()
print("Remaining rows (approx):", remaining_rows)

c:\Users\annik\AppData\Local\Programs\Python\Python313\Lib\site-packages\dask\_task_spec.py:755: FutureWarning: Operation between non boolean Series with different indexes will no longer return a boolean result in a future version. Cast both Series to object type to maintain the prior behavior.
  return self.func(*new_argspec)


Remaining rows (approx): 43845749


In [4]:
# Repartition to avoid memory overflow
filtered_dd = filtered_dd.repartition(npartitions=200)

# Function to fill missing values
def fill_missing_values(df):
    df['details'] = df['details'].fillna("Unknown")
    df['store'] = df['store'].fillna("Unknown")
    return df

# Apply map_partitions to fill missing values
filtered_dd = filtered_dd.map_partitions(fill_missing_values)

**From here is where failed due to RAM**

In [ ]:
from dask import delayed

# Function to count 'Unknown' in a single series
def count_unknown_partition(partition):
    return (partition == "Unknown").sum()

missing_details_brands_total = 0

for delayed_partition in filtered_dd['details'].to_delayed():
    # Wrap the delayed partition with a custom delayed function
    count = delayed(count_unknown_partition)(delayed_partition)
    missing_details_brands_total += count.compute()

print("Missing brands from details (approx):", missing_details_brands_total)

In [ ]:
missing_store_brands_total = 0

for delayed_partition in filtered_dd['store'].to_delayed():
    count = delayed(count_unknown_partition)(delayed_partition)
    missing_store_brands_total += count.compute()

print("Missing brands from store (approx):", missing_store_brands_total)

In [ ]:
merged_dd = merged_dd.drop_duplicates(subset=['user_id', 'asin', 'text'], keep='first')

In [ ]:
def review_length(text):
    if isinstance(text, str):
        return len(re.findall(r'\b\w+\b', text))
    return 0

merged_dd['review_length'] = merged_dd['text'].map(review_length, meta=('text', 'int64'))
print("Review length column created!")

In [ ]:
merged_dd['timestamp'] = dd.to_datetime(merged_dd['timestamp'], errors='coerce')
merged_dd['year'] = merged_dd['timestamp'].dt.year
print("Year column extracted and created!")

In [ ]:
merged_dd.to_parquet("Electronics_Merged.parquet", engine='fastparquet', write_index=False)
print("Saved to Parquet successfully!")